In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

# Text analysis of DOJ press releases (52 points)

For background, here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 

Here's the code the dataset owner used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

In [2]:
## first, unzip the file pset4_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
release = doj.loc[doj['id']=='17-1204']
pharma = release['contents'].to_string()

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `processtext` function here has an example of tokenizing and filtering to words where .isalpha() is true: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/06_textasdata_partII_topicmodeling.ipynb
- Part of speech tagging section of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb



In [4]:
## your code here to restrict to alpha
pharma_processed = " ".join([word for word in word_tokenize(pharma) if word.isalpha()])

In [5]:
## your code here for part of speech tagging and part C
pharma_tags = nltk.pos_tag(word_tokenize(pharma_processed))

adjectives = [word for word in pharma_tags if word[1] == 'JJ']

pharmalist = [(adj[0],adjectives.count(adj)) for adj in adjectives]

pharma_adj_df = pd.DataFrame(pharmalist).drop_duplicates()
pharma_adj_df.columns = ['adjective', 'frequency']
pharma_adj_df.sort_values('frequency', ascending=False).head(5)

,adjective,frequency
10,former,8
33,opioid,5
0,nationwide,4
38,addictive,3
9,other,3


## 1.2 named entity recognition (4 points)

**Resources**:

- For parts A and B: named entity recognition part of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb
- For part E, re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/04_basicregex_solutions.ipynb

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [7]:
## your code here 
pharma_spacy = nlp(pharma)
named_entities = [(token.text, token.label_) for token in pharma_spacy.ents]
# named_entities
LAW_entities = list({entity[0] for entity in named_entities if entity[1]=='LAW'})
LAW_entities

['RICO', 'the Controlled Substances Act']

The Racketeer Influenced and Corrupt Organizations (RICO) Act is a law that allows the US government to prosecute those who commit acts of racketeering (fraudulent business dealings) in order to combat organized crime. 

This act applies to pharmaceutical kickback cases because when a company basically bribes a doctor/healthcare professional to promote a drug to patients often without sound medical reason, it is a business deal that can be considered racketeering.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [8]:
## your code here
label_date = [entity for entity in named_entities if entity[1]=='DATE']
regex = r'.*year(s?).*'
contains_year = [entity[0] for entity in label_date if re.findall(regex, entity[0])]
contains_year

['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). *Hint*: you may want to use re.search or re.findall 


In [9]:
## your code here
original_parts = re.findall('[^.]*year[^.]*\.', pharma)
original_parts

['\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.',
 'The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.',
 '\xa0 The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [10]:
## your code here for subsetting
topics = ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']

doj_subset = doj.loc[doj['topics_clean'].isin(topics)]

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (hint: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- You may want to use `re.escape` at some point to avoid errors relating to escape characters like ( in the press release
- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [13]:
# your code here to define function and execute it

def sentiment(release):
    release_spacy = nlp(release)
    release_named_entities = [token.text for token in release_spacy.ents]

    escaped_entities = [re.escape(entity) for entity in release_named_entities]
    release_regex = '|'.join(escaped_entities)
    no_named_entities = re.sub(release_regex, '', release)
    
    
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sent_obj.polarity_scores(no_named_entities)
    
    return sentiment_dict

all_sentiments = [sentiment(press_release) for press_release in doj_subset['contents']]


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

In [14]:
# all_sentiments
sentiments_df = pd.DataFrame(all_sentiments)
doj_subset_wscore = pd.merge(doj_subset.reset_index(), sentiments_df.reset_index(), left_index=True, right_index=True)
doj_sorted = doj_subset_wscore[['id', 'contents', 'neg']].sort_values('neg', ascending=False).reset_index()
doj_sorted.iloc[0:2]

,index,id,contents,neg
0,13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.321
1,34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.303


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [15]:
# your code here 
doj_subset_wscore[['compound', 'topics_clean']].groupby('topics_clean').agg(np.mean)

,compound
topics_clean,
Civil Rights,-0.101187
Hate Crimes,-0.931808
Project Safe Childhood,-0.662036


These scores are varied because these topics are inherently different and may lend themselves to the use of more negative or more positive language. Civil rights has the least negative compound score of the three topics which makes sense as passages about civil rights may use more hopeful language and talk about the improvement of civil rights. 

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's more condensed code with topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb
- Here's longer code with more broken-out topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/06_textasdata_partII_topicmodeling.ipynb 

In [16]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [17]:
# your code here
custom_doj_stopwords = set(custom_doj_stopwords)
stop_words = set(stopwords.words('english'))
all_stopwords = set.union(stop_words,custom_doj_stopwords)

snowball = SnowballStemmer(language="english")

def process(string):

    string_lower = string.lower()
    tokens = word_tokenize(string_lower)
    tokenize_string = [s for s in tokens if not s.lower() in all_stopwords]
    alpha_string = [re.sub('[^A-Za-z]+', '', s) for s in tokenize_string]
    char_string = [s for s in alpha_string if len(s)>=4]
    stem_string = [snowball.stem(s) for s in char_string]
    final_string = " ".join(stem_string)
    return final_string

    

In [18]:
doj_subset_wscore['processed_text'] = [process(s) for s in doj_subset_wscore['contents']]

In [19]:
two_rows = doj_subset_wscore.loc[(doj_subset_wscore['id']=='16-718') | (doj_subset_wscore['id']=='16-217')]
two_rows[['id', 'contents', 'processed_text']]

,id,contents,processed_text
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv officerinvolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem officerinvolv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc officerinvolv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim officerinvolv shoot effect quick investig officerinvolv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr co

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the create_dtm function: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb


In [20]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(),
                columns=vectorizer.get_feature_names())
    metadata.columns = ["metadata_" + col for col in metadata.columns]
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), 
                                        dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [21]:
# your code here
dtm = create_dtm(list_of_strings= doj_subset_wscore['processed_text'],
                metadata = 
                doj_subset_wscore[['id', 'compound', 'topics_clean']])

top_5 = float(dtm['metadata_compound'].quantile(0.95))
bottom_5 = float(dtm['metadata_compound'].quantile(0.05))

top_5_df = dtm.loc[dtm['metadata_compound']>=top_5]
bottom_5_df = dtm.loc[dtm['metadata_compound']<=bottom_5]
civil_df = dtm.loc[dtm['metadata_topics_clean']=='Civil Rights']
hate_df = dtm.loc[dtm['metadata_topics_clean']=='Hate Crimes']
child_df = dtm.loc[dtm['metadata_topics_clean']=='Project Safe Childhood']

def get_topwords(df):
    topdf = df[[col for col in df.columns
                      if "metadata" not in col and
                      col != "index"]].sum(axis = 0)
    return topdf.sort_values(ascending = False).head(10)

print('Top 10 words for Most Pos 5%')
get_topwords(top_5_df)
print('Top 10 words for Most Neg 5%')
get_topwords(bottom_5_df)
print('Top 10 words for Civil Rights Topic')
get_topwords(civil_df)
print('Top 10 words for Hate Crimes Topic')
get_topwords(hate_df)
print('Top 10 words for Project Safe Childhood Topic')
get_topwords(child_df)

Top 10 words for Most Pos 5%


agreement     175
enforc        117
state         117
ensur         110
disabl        104
communiti      88
servic         88
settlement     87
student        87
court          85
dtype: int64

Top 10 words for Most Neg 5%


assault     190
crime       172
victim      166
offic       155
hate        130
defend      124
sentenc     102
anderson     94
charg        93
feder        92
dtype: int64

Top 10 words for Civil Rights Topic


offic        632
hous         626
discrimin    549
enforc       533
disabl       520
said         497
violat       476
feder        475
state        446
agreement    410
dtype: int64

Top 10 words for Hate Crimes Topic


victim      591
crime       550
hate        488
prosecut    478
defend      459
sentenc     455
charg       452
guilti      430
feder       426
said        424
dtype: int64

Top 10 words for Project Safe Childhood Topic


child          1019
exploit         699
sexual          570
safe            476
project         472
childhood       472
pornographi     448
children        422
crimin          404
prosecut        374
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [22]:
# your code here
text_raw_tokens = [wordpunct_tokenize(one_text) 
                for one_text in 
                doj_subset_wscore['processed_text']]

text_raw_dict = corpora.Dictionary(text_raw_tokens)

corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                num_topics = 3, id2word=text_raw_dict, 
                                passes=6, alpha = 'auto',
                                per_word_topics = True, random_state = 2)

topics = ldamod.print_topics(num_words = 15)

for topic in topics:
    print(topic)

(0, '0.011*"hous" + 0.010*"disabl" + 0.010*"discrimin" + 0.007*"enforc" + 0.007*"agreement" + 0.007*"state" + 0.007*"alleg" + 0.006*"court" + 0.006*"said" + 0.006*"feder" + 0.006*"fair" + 0.005*"settlement" + 0.005*"requir" + 0.005*"general" + 0.005*"violat"')
(1, '0.010*"victim" + 0.009*"charg" + 0.009*"crime" + 0.008*"defend" + 0.008*"prosecut" + 0.008*"said" + 0.008*"sentenc" + 0.008*"feder" + 0.007*"hate" + 0.007*"guilti" + 0.007*"assault" + 0.006*"indict" + 0.006*"offic" + 0.006*"investig" + 0.006*"prison"')
(2, '0.020*"child" + 0.013*"exploit" + 0.011*"sexual" + 0.010*"prosecut" + 0.009*"sentenc" + 0.009*"safe" + 0.009*"childhood" + 0.009*"crimin" + 0.009*"project" + 0.009*"victim" + 0.009*"pornographi" + 0.009*"children" + 0.008*"guilti" + 0.007*"investig" + 0.007*"feder"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb
- If you're getting errors, use shape, len, and other commands to check the dimensionality of things at different steps 

In [23]:
## your code here to get doc-level topic probabilities 
topic_probs = [ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
# topic_probs
assert len(topic_probs)==len(doj_subset_wscore.index)

In [24]:
## your code here to add those topic probabilities to the dataframe
# topic_probs_df
topic_only_probs = [[tuple[1] for tuple in list] for list in topic_probs]
topic_probs_df = pd.DataFrame(topic_only_probs)

topic_probs_df.columns = ['topic_1', 'topic_2', 'topic_3']
# topic_probs_df
topic_model_df = pd.merge(doj_subset_wscore, topic_probs_df, left_index=True, right_index=True)

# topic_model_df[['id', 'topic_1', 'topic_2', 'topic_3']]

topic_model_df['top_topic'] = topic_model_df[['topic_1', 'topic_2', 'topic_3']].idxmax(axis=1)
# topic_model_df

In [25]:
## your code here to summarize the topic proportions for each of the topics_clean 
# pd.crosstab(df.make, df.num_doors, margins=True, margins_name="Total")
normalized = pd.crosstab(index=topic_model_df['topics_clean'], columns=topic_model_df['top_topic'])

normalized[['topic_1', 'topic_2', 'topic_3']] = [normalized['topic_1'] / (normalized['topic_1']+normalized['topic_2']+normalized['topic_3']),
                                                 normalized['topic_2'] / (normalized['topic_1']+normalized['topic_2']+normalized['topic_3']),
                                                 normalized['topic_3'] / (normalized['topic_1']+normalized['topic_2']+normalized['topic_3'])]

normalized

top_topic,topic_1,topic_2,topic_3
topics_clean,,,
Civil Rights,0.662295,0.008130,0.006024
Hate Crimes,0.327869,0.857724,0.000000
Project Safe Childhood,0.009836,0.134146,0.993976


The reason that some fit more cleanly than others into the estimated topics is because there are passages such 17-271 that mostly use language that clearly pertains to one estimated topic (Civil Rights), meanwhile there are passages like 18-359 which use language that might be associated with another topic. For example, 18-359 often discusses sexual harassment, but is categorized as Civil Rights. The word "sexual" is one of the most used words in the Project Safe Childhood topic, which will make it fit partially with "Civil Rights" and "Project Safe Childhood" topics.

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [26]:
## your code here
def create_bigram_onedoc(txt):
    txt = str(txt)
    tokens = word_tokenize(txt)
    bigrams = [first+'_'+second for first, second in zip(tokens, tokens[1:])]
    bigrams
    bigrams_text = ' '.join(bigrams)
    return bigrams_text

doj_subset_wscore['processed_text_bigrams'] = [create_bigram_onedoc(txt) for txt in doj_subset_wscore['processed_text']]

In [27]:
doj_subset_wscore[['id', 'processed_text', 'processed_text_bigrams']].loc[doj_subset_wscore['id']=='16-217']

,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv officerinvolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem officerinvolv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc officerinvolv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim officerinvolv shoot effect quick investig officerinvolv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_officerinvolv officerinvolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_officerinvolv officerinvolv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_officerinvolv officerinvolv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_officerinvolv officerinvolv_shoot shoot_effect effect_quick quick_investig investig_officerinvolv officerinvolv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief 

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [28]:
# your code here 
dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore['processed_text_bigrams'],
                metadata = 
                doj_subset_wscore[['id', 'compound', 'topics_clean']])

dtm.shape
dtm_bigram.shape

(717, 7417)

(717, 73245)

dtm_bigram has more dimensions because there are more unique pairs of words than there are unique single words.

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [29]:
# your code here
civil_bigram_df = dtm_bigram.loc[dtm_bigram['metadata_topics_clean']=='Civil Rights']
hate_bigram_df = dtm_bigram.loc[dtm_bigram['metadata_topics_clean']=='Hate Crimes']
child_bigram_df = dtm_bigram.loc[dtm_bigram['metadata_topics_clean']=='Project Safe Childhood']

print('Top 10 bigrams for Civil Rights Topic')
get_topwords(civil_bigram_df)
print('Top 10 bigrams for Hate Crimes Topic')
get_topwords(hate_bigram_df)
print('Top 10 bigrams for Project Safe Childhood Topic')
get_topwords(child_bigram_df)

Top 10 bigrams for Civil Rights Topic


fair_hous         228
princip_deputi    221
deputi_general    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        154
nation_origin     142
consent_decre     128
dtype: int64

Top 10 bigrams for Hate Crimes Topic


hate_crime       373
plead_guilti     275
year_prison      160
special_agent    117
thoma_perez      111
grand_juri       101
perez_general     95
said_thoma        91
unit_state        88
act_general       85
dtype: int64

Top 10 bigrams for Project Safe Childhood Topic


project_safe         472
safe_childhood       472
child_pornographi    446
child_exploit        279
sexual_exploit       223
exploit_children     200
plead_guilti         197
exploit_obscen       175
child_sexual         174
obscen_section       174
dtype: int64

# 4. Optional extra credit 1 (3 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [23]:
# your code here 

# 5. Optional extra credit 2 (6 points)

Review the scraping code here: https://github.com/jbencina/dojreleases/blob/master/scraper.py
    
Write code to scrape press releases from more recent years from the DOJ website than the years available in the combined.json (which stops around 2018 or 2019) and produce a visualization of how top words or topics in the press releases changed across the Trump administration (2016-December 2020) versus Biden administration (January 2021 onwards)  

In [24]:
# your code here